In [5]:
import re
import pandas as pd
from numpy.linalg import norm
from profanity import profanity
import csv
import numpy as np
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
from collections import Counter
import nltk
import spacy
from textblob import TextBlob
nlp = spacy.load('de_core_news_sm')
RAW_DATA_PATH = 'data/raw/'

Helper Functions

In [6]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [7]:
def find_longest_word(word_list):
    longest_word =  max(word_list, key=len)
    return len(longest_word)

Data Extraction

In [8]:
df_Lied = pd.read_csv(RAW_DATA_PATH + 'LIED.csv', usecols=['ID','INTERPRET', 'TITEL', 'SPRACHE_DEUTSCH', 'TEXT_TEIL1', 'TEXT_TEIL2', 'TEXT_TEIL3', 'TEXT_TEIL4'])
print(df_Lied.head())

df_Chart_Position = pd.read_csv(RAW_DATA_PATH + 'CHART_POSITION.csv', usecols=['LIED_ID', 'POSITION', 'DATUM_VON', 'DATUM_BIS'])
print(df_Chart_Position.head())

with open(RAW_DATA_PATH+'Stoppwords.csv', newline='', encoding='UTF-8') as f:
    stopwords_list = list(csv.reader(f))
stopwords_list = [word.upper() for word in flatten(stopwords_list)]
print(type(stopwords_list))

        ID INTERPRET    TITEL SPRACHE_DEUTSCH  \
0  1148163     Yasha  Strand               Ja   
1  1148163     Yasha  Strand               Ja   
2  1148163     Yasha  Strand               Ja   
3  1148163     Yasha  Strand               Ja   
4  1148163     Yasha  Strand               Ja   

                                          TEXT_TEIL1 TEXT_TEIL2 TEXT_TEIL3  \
0  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...        NaN        NaN   
1  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...        NaN        NaN   
2  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...        NaN        NaN   
3  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...        NaN        NaN   
4  LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN...        NaN        NaN   

  TEXT_TEIL4  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  
   LIED_ID  POSITION   DATUM_VON   DATUM_BIS
0    26024        26  1983-04-18  1983-04-24
1     1073        46  1984-06-04  1984-06-10
2     1131 

Data Conversion

In [9]:
df_Lied['Text'] = df_Lied['TEXT_TEIL1'].fillna('') + df_Lied['TEXT_TEIL2'].fillna('') + df_Lied['TEXT_TEIL3'].fillna('') + df_Lied['TEXT_TEIL4'].fillna('')
#df_Lied.drop(columns=['TEXT_TEIL1', 'TEXT_TEIL2', 'TEXT_TEIL3', 'TEXT_TEIL4'], axis=1, inplace=True)

df_Chart_Position['DATUM_VON'] = pd.to_datetime(df_Chart_Position['DATUM_VON'])
df_Chart_Position['DATUM_BIS'] = pd.to_datetime(df_Chart_Position['DATUM_BIS'])
df_Chart_Position['DAUER'] = (df_Chart_Position['DATUM_BIS'] - df_Chart_Position['DATUM_VON']).dt.days.astype('int16')
df_Chart_Position['Jahr'] = df_Chart_Position['DATUM_BIS'].dt.year.astype('int16')
df_Chart_Position['Monat'] =  df_Chart_Position['DATUM_BIS'].dt.month.astype('int16')

#df_Lied['TEXT_LAENGE'] = list(len(word) for word in df_Lied['Text'])
df_Lied['ANZ_UNIQUE_WOERTER'] = list(len(set(word.split(" "))) for word in df_Lied['Text'])
df_Lied['LAENGE_LAENGSTES_WORT'] = list(len(max(set(word.split(" ")), key=len)) for word in df_Lied['Text'])
df_Lied[['LAENGE_LAENGSTES_WORT','ID']].to_csv('Data/processed/Wort_LAENGE_v2.csv')
df_Lied['INTERPRET'].value_counts(ascending=True, dropna=True, normalize=True)\
    #.to_csv('Data/processed/Interpret_Occurrences_normalised.csv')

MAX_RANK = 50
df_Chart_Position['RANK_SCORE'] = MAX_RANK - df_Chart_Position['POSITION'] + 1

#df_copy_Chart_Position = df_Chart_Position["LIED_ID"]

#df_Chart_Position_Duplicates = df_Chart_Position[df_copy_Chart_Position.isin(df_copy_Chart_Position[df_copy_Chart_Position.duplicated()])].sort_values("LIED_ID")

In [10]:
print(df_Lied['Text'][1000])

EY  SHAWTY WILL ALLEINE  SEIN  ALLEIN  DENN SIE HAT GERADE BAD VIBES SHAWTY WILL ALLEINE  SEIN  ALLEIN  IST NICHT ERREICHBAR AUF FACETIME  YEAH  MIR GENUEGT SCHON EIN CALL VON DIR  BABY  VON DIR  BABY  KEINER WEISS  WIESO  DOCH DU MACHST MICH SO CRAZY  KEINER WEISS  WIESO  EY  SHAWTY WILL ALLEINE  SEIN  ALLEIN  DENN SIE HAT GERADE BAD VIBES SIE HAT BAD VIBES  BAD VIBES   ES IST LATE NIGHT  NIGHT  ERFUELL IHR JEDEN WUNSCH  DENN WIR LEBEN DAS FAST LIFE  OH  JA  ICH WEISS  ALLE MACHEN AUGE AUF MEIN BABY  BABY  GIB MIR DEIN WORT UND ICH MACH DICH ZU MEINER LADY  JA  VERMISS DICH SCHON  JA  SOBALD ICH AUS DEM HAUS LAUF  OH OH OH  DU FICKST MEIN  N KOPF IRGENDWIE  ABER ICH STEH DRAUF  ICH STEH DRAUF  DU GEHST MIR AUS DEM WEG UND SUCHST DANN WIEDER NAEHE  OH YEAH  WAS DU DIR DABEI DENKST  DAS MUSS MAN NICHT VERSTEHEN MIT DIR FUEHL ICH MICH  ALS WAER ICH IM FINALE  FINALE  SCHEISS AUF ALLES  BRAUCH NUR DICH  KEINE POKALE  POKALE  DU REDEST HIN UND HER UND SENDEST MIR SIGNALE  OHH  DASS DU MICH

Text Preprocessing

In [12]:
#lemmatization

#df_Lied['Text'].apply(lambda x: " ".join([word.lemma_ for text in df_Lied['Text'] for word in nlp(text)]))

# tokenize
df_Lied['processed_text'] = df_Lied.apply(lambda row: nltk.word_tokenize(row['Text']), axis=1)

#remove stopwords
df_Lied['processed_text'] = df_Lied['processed_text'].apply(lambda x: [item for item in x if item not in stopwords_list])

#remove numbers
df_Lied['processed_text'].apply(lambda x : re.sub('\w*\d\w*','', x))
#print(df_Lied.head())

#Stemming


# ngram


TypeError: expected string or bytes-like object, got 'list'

In [111]:
print(df_Lied['tokenized_text'][1000])

['EY', 'SHAWTY', 'WILL', 'ALLEINE', 'ALLEIN', 'GERADE', 'BAD', 'VIBES', 'SHAWTY', 'WILL', 'ALLEINE', 'ALLEIN', 'ERREICHBAR', 'FACETIME', 'YEAH', 'GENUEGT', 'SCHON', 'CALL', 'BABY', 'BABY', 'WEISS', 'DOCH', 'MACHST', 'CRAZY', 'WEISS', 'EY', 'SHAWTY', 'WILL', 'ALLEINE', 'ALLEIN', 'GERADE', 'BAD', 'VIBES', 'BAD', 'VIBES', 'BAD', 'VIBES', 'LATE', 'NIGHT', 'NIGHT', 'ERFUELL', 'WUNSCH', 'LEBEN', 'FAST', 'LIFE', 'OH', 'JA', 'WEISS', 'MACHEN', 'AUGE', 'BABY', 'BABY', 'GIB', 'MACH', 'LADY', 'JA', 'VERMISS', 'SCHON', 'JA', 'HAUS', 'LAUF', 'OH', 'OH', 'OH', 'FICKST', 'KOPF', 'IRGENDWIE', 'STEH', 'DRAUF', 'STEH', 'DRAUF', 'GEHST', 'WEG', 'SUCHST', 'WIEDER', 'NAEHE', 'OH', 'YEAH', 'DABEI', 'DENKST', 'MUSS', 'VERSTEHEN', 'FUEHL', 'WAER', 'FINALE', 'FINALE', 'SCHEISS', 'BRAUCH', 'NUR', 'POKALE', 'POKALE', 'REDEST', 'HIN', 'HER', 'SENDEST', 'SIGNALE', 'OHH', 'ZAPPELN', 'LAESST', 'WISSEN', 'DOCH', 'SHAWTY', 'WILL', 'ALLEINE', 'ALLEIN', 'GERADE', 'BAD', 'VIBES', 'SHAWTY', 'WILL', 'ALLEINE', 'ERREICHBAR'

profanity check

In [112]:
with open(RAW_DATA_PATH+'german-profanity-list.csv', newline='', encoding='UTF-8') as f:
    profanity_list = list(csv.reader(f))

profanity.load_words(flatten(profanity_list))
profanity.contains_profanity(df_Lied['Text'][0])

False

Percentage of Stopwords

In [18]:
df_Lied['Number_of_Stopwords'] = df_Lied['Text'].str.split().apply(lambda x: len(set(x) & set(stopwords_list)))
df_Lied['Stopword_Percentage'] = df_Lied.Number_of_Stopwords.apply(lambda row: row/len(df_Lied['Text']))
df_Lied[['ID','Number_of_Stopwords','Stopword_Percentage']].to_csv('Data/processed/Stoppwords.csv')

In [20]:
df_Chart_Position.head()

,LIED_ID,POSITION,DATUM_VON,DATUM_BIS,DAUER,Jahr,Monat,RANK_SCORE
0,26024,26,1983-04-18,1983-04-24,6,1983,4,25
1,1073,46,1984-06-04,1984-06-10,6,1984,6,5
2,1131,33,1984-06-18,1984-06-24,6,1984,6,18
3,15819,22,1984-08-13,1984-08-19,6,1984,8,29
4,7324,6,1984-08-27,1984-09-02,6,1984,9,45


Sentiment Analysis

LDA (BoW) - Title

Named Entity Recognition

Readability analysis

Title Analysis

In [240]:
titel_list = [word.upper() for word in list(set(df_Lied['TITEL']))]
tokenizer = RegexpTokenizer(r'\w+')
tokenized_list = [word for titel in titel_list for word in tokenizer.tokenize(titel)]
no_stopwords_title_list = [item for item in tokenized_list if item not in stopwords_list]
df_title_word_occurrence = pd.value_counts(np.array(no_stopwords_title_list))
df_title_word_occurrence.to_csv('Data/processed/title_word_occurrence.csv')

Seasonal determination

Boosted Words -Analysis

Merge df

In [ ]:
#df = pd.merge(df_Lied, df_Chart_Position.set_axis(['DATUM_VON','DATUM_BIS', 'ID', 'POSITION', 'DAUER', 'BELIEBTHEIT', 'Jahr', 'Monat'], axis=1, inplace=False), on='ID', how='inner').sort_values(by=['ID'])
print(df_Lied.isnull().values.any())#df muss noch ausgetauscht werden

Label Smoothing

Normalising

EDA